In [6]:
import numpy as np
from math import log,sqrt,exp
from time import time
from random import gauss,seed
import pandas as pd

In [7]:
no_sim = [10000,20000,30000,40000]

In [9]:
# MCS valuation of European call option with Numpy
np.random.seed(20000)
t0 = time()

In [11]:
# Parameters
S0 = 100; K = 100; T = 1.0; r = 0.05; sigma = 0.2
M = 50; dt = T/M; #I = 10000
results_prices = []
results_se = []
for I in no_sim:
    results_prices_I = []
    results_se_I = []
    # Simulating I Paths with M times Steps using GBM exact solution!
    S = np.zeros((M+1,I))
    S[0] = S0
    for t in range(1,M+1):
        z = np.random.standard_normal(I) # Psedu random numbers
        S[t] = S[t-1]*np.exp((r-0.5*sigma**2)*dt+sigma*sqrt(dt)*z)
        # Vectorized operation per time step over all paths
    #Calculating the MCS estimators
    C0_gbm = exp(-r*t)*np.sum(np.maximum(S[-1]-K,0))/I
    C0_gbm_se = exp(-r*t)*np.std(np.maximum(S[-1]-K,0))/np.sqrt(I)
    results_prices_I.append(C0_gbm)
    results_se_I.append(C0_gbm_se)
    
    # Simulating I paths with M time steps using Euler_Maruyama Method:
    S=np.zeros((M+1,I))
    S[0] = S0
    for t in range(1,M+1):
        z = np.random.standard_normal(I) # Psedu random numbers
        S[t] = S[t-1]*(1+r*dt+sigma*sqrt(dt)*z)
         # Vectorized operation per time step over all paths
    #Calculating the MCS estimators
    C0_EM = exp(-r*t)*np.sum(np.maximum(S[-1]-K,0))/I
    C0_EM_se = exp(-r*T)*np.std(np.maximum(S[-1]-K,0))/np.sqrt(I)
    results_prices_I.append(C0_EM)
    results_se_I.append(C0_EM_se)
    
    #Simulating I paths with M time steps using Milstein method:
    S = np.zeros((M+1,I))
    S[0] = S0
    for t in range(1,M+1):
        z = np.random.standard_normal(I) # Psedu random numbers
        S[t] = S[t-1]*(1+r*dt+sigma*sqrt(dt)*z + 0.5*sigma**2*((sqrt(dt)*z)**2-dt))
         # Vectorized operation per time step over all paths
    #Calculating the MCS estimators
    C0_MS = exp(-r*t)*np.sum(np.maximum(S[-1]-K,0))/I
    C0_MS_se = exp(-r*T)*np.std(np.maximum(S[-1]-K,0))/np.sqrt(I)
    results_prices_I.append(C0_MS)
    results_se_I.append(C0_MS_se)
    
    results_prices.append(results_prices_I)
    results_se.append(results_se_I)

results_prices_df = pd.DataFrame(results_prices,columns=['GBM','Euler-Maruyama','Milstein'],index=
                                ['sim 10,000','sim 20,000','sim 30,000','sim 40,000'])

print("########################## Europena Option Estimation Values for GBM,Euler-Maruyama & Milstein #################")
print(results_prices_df)
    
results_SE_df = pd.DataFrame(results_se,columns=['GBM','Euler-Maruyama','Milstein'],index=
                                ['sim 10,000','sim 20,000','sim 30,000','sim 40,000'])

print("########################## Calculated MSE for GBM,Euler-Maruyama & Milstein #################")
print(results_SE_df)

########################## Europena Option Estimation Values for GBM,Euler-Maruyama & Milstein #################
                 GBM  Euler-Maruyama  Milstein
sim 10,000  0.896641        0.911223  0.910896
sim 20,000  0.909789        0.902160  0.904427
sim 30,000  0.910706        0.910795  0.899826
sim 40,000  0.902655        0.904514  0.899847
########################## Calculated MSE for GBM,Euler-Maruyama & Milstein #################
                 GBM  Euler-Maruyama  Milstein
sim 10,000  0.012612        0.147617  0.147926
sim 20,000  0.009016        0.104591  0.105254
sim 30,000  0.007416        0.085222  0.084567
sim 40,000  0.006352        0.073958  0.073421
